# Google trace analysis colab

This colab provides several example queries and graphs using [Altair](https://altair-viz.github.io/) for the 2019 Google cluster trace. Further examples will be added over time.

**Important:** in order to be able to run the queries you will need to:

1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


In [ ]:
#@title Please input your project id
import pandas as pd
import numpy as np
from tqdm import tqdm
import altair as alt
from google.cloud import bigquery
# Provide credentials to the runtime
from google.oauth2 import service_account
# from google.cloud.bigquery import magics

# Path to your service account key file
service_account_file = '~/credentials.json'

# Create credentials object
credentials = service_account.Credentials.from_service_account_file(service_account_file)

# Initialize the BigQuery client with the credentials
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

Authenticated


In [ ]:
#Loading Data for collection events
sql_query = ('''
    SELECT
        time,
        collection_id,
        type,
        user,
        parent_collection_id,
        start_after_collection_ids,
        ROW_NUMBER() OVER (PARTITION BY user ORDER BY time) as user_row_num
    FROM `google.com:google-cluster-data`.clusterdata_2019_a.collection_events
    LIMIT 10000000
''')

# Convert the query result to a DataFrame
df = client.query(sql_query).to_dataframe()
df['time'] = pd.to_numeric(df['time'], errors='coerce').astype(float)
df['collection_id'] = pd.to_numeric(df['collection_id'], errors='coerce').astype(float)
df.head()

,time,collection_id,type,user,parent_collection_id,start_after_collection_ids,user_row_num
0,1.380814e+11,3.767278e+11,5,9o60U7n4O4ZM5KJVV4DJRwG/waL4/H+TtliY0UwiQSg=,<NA>,[],37
1,3.662464e+11,3.794910e+11,5,9o60U7n4O4ZM5KJVV4DJRwG/waL4/H+TtliY0UwiQSg=,<NA>,[],130
2,1.751696e+12,3.927326e+11,5,9o60U7n4O4ZM5KJVV4DJRwG/waL4/H+TtliY0UwiQSg=,<NA>,[],264
3,2.557382e+12,3.998548e+11,5,9o60U7n4O4ZM5KJVV4DJRwG/waL4/H+TtliY0UwiQSg=,<NA>,[],336
4,2.755679e+11,3.786410e+11,5,GrtqGa19qo7FbXQuKxquCensI5xGOTDgsxb1AgoCmts=,<NA>,[],217


In [ ]:
sql_usage = ('''SELECT *
                FROM `google.com:google-cluster-data`.clusterdata_2019_a.instance_usage
                LIMIT 10''')

df_usage = client.query(sql_usage).to_dataframe()
df_usage.head()

,start_time,end_time,collection_id,instance_index,machine_id,alloc_collection_id,alloc_instance_index,collection_type,average_usage,maximum_usage,random_sample_usage,assigned_memory,page_cache_memory,cycles_per_instruction,memory_accesses_per_instruction,sample_rate,cpu_usage_distribution,tail_cpu_usage_distribution
0,24600000000,24900000000,372912766464,13,39175913234,372912766423,3,0,"{'cpus': 0.0045318603515625, 'memory': 0.00299...","{'cpus': 0.031982421875, 'memory': 0.003398895...","{'cpus': 0.0008831024169921875, 'memory': None}",0.0,0.000037,1.632373,0.001643,1.000000,"[8.7738037109375e-05, 0.00011348724365234375, ...","[0.01611328125, 0.017364501953125, 0.019927978..."
1,1608000000000,1608300000000,124264792320,2,1715246892,124264764991,2,0,"{'cpus': 0.000225067138671875, 'memory': 0.000...","{'cpus': 0.0007314682006835938, 'memory': 0.00...","{'cpus': 0.0001888275146484375, 'memory': None}",0.0,0.000239,4.958133,0.015517,1.000000,"[0.0001697540283203125, 0.0001811981201171875,...","[0.0002956390380859375, 0.0003147125244140625,..."
2,1159200000000,1159500000000,124264792320,0,21263130,124264764991,0,0,"{'cpus': 0.00020122528076171875, 'memory': 0.0...","{'cpus': 0.0014553070068359375, 'memory': 0.00...","{'cpus': 0.00024890899658203125, 'memory': None}",0.0,0.000209,3.070624,0.010065,1.000000,"[0.00014019012451171875, 0.0001459121704101562...","[0.00029754638671875, 0.00031948089599609375, ..."
3,1290900000000,1291200000000,124264792320,2,1715246892,124264764991,2,0,"{'cpus': 0.00029468536376953125, 'memory': 0.0...","{'cpus': 0.004730224609375, 'memory': 0.000576...","{'cpus': 0.00026607513427734375, 'memory': None}",0.0,0.000218,NaN,NaN,1.000000,"[0.00017070770263671875, 0.0001859664916992187...","[0.000347137451171875, 0.000370025634765625, 0..."
4,370500000000,370800000000,124264792320,0,92117608309,124264764991,0,0,"{'cpus': 0.00024127960205078125, 'memory': 0.0...","{'cpus': 0.00091552734375, 'memory': 0.0006847...","{'cpus': 0.0002231597900390625, 'memory': None}",0.0,0.000100,3.083541,0.022465,0.993333,"[0.00018787384033203125, 0.0001964569091796875...","[0.00030517578125, 0.00031566619873046875, 0.0..."


In [ ]:
# Get the unique values in the 'user' column
unique_users = df_usage['collection_id'].unique()
print(len(unique_users))

23836


In [ ]:
# Sort df
sorted_df = df.sort_values(by='collection_type',ascending=False)
print(sorted_df)

In [ ]:
# Use Pandas groupby to classify the tasks by user
grouped_users = df.groupby('user')

# Create a dictionary to hold subtables for each user
subtables = {user: group for user, group in grouped_users}

In [ ]:
print(len(unique_users))
print(subtables[unique_users[2]].head())

1163
             time  type  collection_id  scheduling_class  missing_type  \
216  3.101762e+11     3   3.787515e+11                 1          <NA>   
217  1.159395e+12     0   3.835821e+11                 1          <NA>   
218  2.376499e+12     2   3.989684e+11                 2          <NA>   
219  1.666261e+12     0   3.855788e+11                 3          <NA>   
220  8.500808e+11     3   3.821581e+11                 3          <NA>   

     collection_type  priority  alloc_collection_id  \
216                0       119                 <NA>   
217                0       119                 <NA>   
218                0       119                 <NA>   
219                0       200          33959190870   
220                0       200          33959190870   

                                             user  \
216  +9EArMpTcRKY6SZ+doAw1QllovlleQnzrJD843XiIDo=   
217  +9EArMpTcRKY6SZ+doAw1QllovlleQnzrJD843XiIDo=   
218  +9EArMpTcRKY6SZ+doAw1QllovlleQnzrJD843XiIDo=   
219  +9

In [ ]:
# Function to group related tasks based on parent_collection_id and start_after_collection_ids
def group_related_tasks(df):
    grouped_tasks = []  # List to store grouped tasks
    visited = set()  # Set to track visited tasks by their collection IDs

    # Function to find all related tasks for a given collection ID
    def find_related_tasks(collection_id, df):
        related_tasks = set()  # Set to store related collection IDs
        stack = [collection_id]  # Use stack for depth-first search

        while stack:
            current_collection_id = stack.pop()
            # print('currid',current_collection_id)
            if current_collection_id not in visited:
                visited.add(current_collection_id)
                related_tasks.add(current_collection_id)

                # Find tasks where current_collection_id is either the parent_collection_id or in start_after_collection_ids
                parent_related = df[df['parent_collection_id'] == current_collection_id]['collection_id'].tolist()

                # Handle case where start_after_collection_ids is not empty or None
                start_after_related = df[df['start_after_collection_ids'].apply(lambda ids: isinstance(ids, list) and current_collection_id in ids)]['collection_id'].tolist()

                # Add related tasks to the stack for further exploration, if they haven't been visited yet
                for task in parent_related + start_after_related:
                    if task not in visited:
                        stack.append(task)

                # Also, find the parent and start_after of the current task and process them
                parent_task = df[df['collection_id'] == current_collection_id]['parent_collection_id'].tolist()
                start_after_task = df[df['collection_id'] == current_collection_id]['start_after_collection_ids'].tolist()

                # print('par',parent_task)
                # print('start',start_after_task)
                # Modify this condition to handle empty lists of lists
                if isinstance(start_after_task, list) and len(start_after_task) > 0:  # Check if it's a valid list with non-empty elements
                  for task_list in start_after_task:
                      if isinstance(task_list, list) and len(task_list) > 0:  # Only process non-empty lists
                          for task in task_list:
                              if task not in visited:
                                  stack.append(task)

                if not pd.isna(parent_task).all():  # Check if parent_task is not NaN
                    for task in parent_task:
                        if task not in visited:
                            stack.append(task)

        return related_tasks

    # Iterate through all tasks and group them by relationships
    for collection_id in df['collection_id']:
        if collection_id not in visited:
            # Find all related tasks for the current collection_id
            related_tasks = find_related_tasks(collection_id, df)
            # Create a DataFrame for the related tasks and append to grouped_tasks
            grouped_tasks.append(df[df['collection_id'].isin(related_tasks)])

    return grouped_tasks

In [ ]:
# Create a dictionary to store the grouped tasks for each user
grouped_subtables = {}

# Assuming subtables is a dictionary where each user has a corresponding DataFrame
for user, subtable in tqdm(subtables.items(), desc="Processing users"):
    grouped_subtables[user] = group_related_tasks(subtable)

# Now the grouped tasks for each user are stored in the grouped_subtables dictionary

Processing users:   4%|▎         | 69/1949 [10:50<3:05:40,  5.93s/it]

# Basic queries (Don't use! Not Accessible)

This section shows the most basic way of querying the trace using the [bigquery magic](https://googleapis.dev/python/bigquery/latest/magics.html)

In [ ]:
%%bigquery
SELECT count(*)
FROM `google.com:google-cluster-data`.clusterdata_2019_a.machine_events


ERROR:
 404 POST https://bigquery.googleapis.com/bigquery/v2/projects//jobs?prettyPrint=false: Request couldn't be served.

Location: None
Job ID: ce6e80ec-68ff-4b7c-a1fd-c65a5d271a46



In [ ]:
%%bigquery
SELECT COUNT(DISTINCT collection_id) AS collections FROM
`google.com:google-cluster-data`.clusterdata_2019_a.collection_events;


ERROR:
 404 POST https://bigquery.googleapis.com/bigquery/v2/projects//jobs?prettyPrint=false: Request couldn't be served.

Location: None
Job ID: 55e791d5-ef94-417f-bd38-76a4ea4ff478



# Cell level resource usage time series

This query takes a cell as input and plots a resource usage time-series for every hour of the trace broken down by tier.

In [ ]:
#@title Select a cell and a resource to plot the cell level usage series

def query_cell_capacity(cell):
  return '''
SELECT SUM(cpu_cap) AS cpu_capacity,
  SUM(memory_cap) AS memory_capacity
FROM (
  SELECT machine_id, MAX(capacity.cpus) AS cpu_cap,
    MAX(capacity.memory) AS memory_cap
  FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.machine_events
  GROUP BY 1
)
  '''.format(cell=cell)

def query_per_instance_usage_priority(cell):
  return '''
SELECT u.time AS time,
  u.collection_id AS collection_id,
  u.instance_index AS instance_index,
  e.priority AS priority,
  CASE
    WHEN e.priority BETWEEN 0 AND 99 THEN '1_free'
    WHEN e.priority BETWEEN 100 AND 115 THEN '2_beb'
    WHEN e.priority BETWEEN 116 AND 119 THEN '3_mid'
    ELSE '4_prod'
  END AS tier,
  u.cpu_usage AS cpu_usage,
  u.memory_usage AS memory_usage
FROM (
  SELECT start_time AS time,
    collection_id,
    instance_index,
    machine_id,
    average_usage.cpus AS cpu_usage,
    average_usage.memory AS memory_usage
  FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.instance_usage
  WHERE (alloc_collection_id IS NULL OR alloc_collection_id = 0)
    AND (end_time - start_time) >= (5 * 60 * 1e6)
) AS u JOIN (
  SELECT collection_id, instance_index, machine_id,
    MAX(priority) AS priority
  FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.instance_events
  WHERE (alloc_collection_id IS NULL OR alloc_collection_id = 0)
  GROUP BY 1, 2, 3
) AS e ON u.collection_id = e.collection_id
  AND u.instance_index = e.instance_index
  AND u.machine_id = e.machine_id
  '''.format(cell=cell)

def query_per_tier_utilization_time_series(cell, cpu_capacity, memory_capacity):
  return '''
SELECT CAST(FLOOR(time/(1e6 * 60 * 60)) AS INT64) AS hour_index,
  tier,
  SUM(cpu_usage) / (12 * {cpu_capacity}) AS avg_cpu_usage,
  SUM(memory_usage) / (12 * {memory_capacity}) AS avg_memory_usage
FROM ({table})
GROUP BY 1, 2 ORDER BY hour_index
  '''.format(table=query_per_instance_usage_priority(cell),
             cpu_capacity=cpu_capacity, memory_capacity=memory_capacity)

def run_query_utilization_per_time_time_series(cell):
  cell_cap = client.query(query_cell_capacity(cell)).to_dataframe()
  query = query_per_tier_utilization_time_series(
      cell,
      cell_cap['cpu_capacity'][0],
      cell_cap['memory_capacity'][0])
  time_series = client.query(query).to_dataframe()
  return time_series

cell = 'c' #@param ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
hourly_usage = run_query_utilization_per_time_time_series(cell)

# CPU graph
cpu = alt.Chart(hourly_usage).mark_area().encode(
        alt.X('hour_index:N'),
        alt.Y('avg_cpu_usage:Q'),
        color=alt.Color('tier', legend=alt.Legend(orient="left")),
        order=alt.Order('tier', sort='descending'),
        tooltip=['hour_index', 'tier', 'avg_cpu_usage']
  )
cpu.encoding.x.title = "Hour"
cpu.encoding.y.title = "Average CPU usage"
cpu.display()

# Memory graph
memory = alt.Chart(hourly_usage).mark_area().encode(
        alt.X('hour_index:N'),
        alt.Y('avg_memory_usage:Q'),
        color=alt.Color('tier', legend=alt.Legend(orient="left")),
        order=alt.Order('tier', sort='descending'),
        tooltip=['hour_index', 'tier', 'avg_memory_usage']
  )
memory.encoding.x.title = "Hour"
memory.encoding.y.title = "Average memory usage"
memory.display()

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

alt.Chart(...)

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


#Per machine resource usage distribution

This query takes a cell as input and plots a per-machine resource utilization CDF.

In [ ]:
#@title Select a cell and plot its per-machine resource utilization CDFs

# Functions to plot CDFs using Altair
def pick_quantiles_from_tall_dataframe(data, qcol, name=""):
  quantiles = pd.DataFrame([x for x in data[qcol]]).transpose()
  if name != "":
    quantiles.columns = data[name]
  return quantiles

# - data: a dataframe with one row and one or more columns of quantiles (results
#   returned from APPROX_QUANTILES)
# - qcols: a list of names of the quantiles
# - names: the names of each returned quantiles' columns.
def pick_quantiles_from_wide_dataframe(data, qcols, names=[]):
  quantiles = {}
  i = 0
  for qcol in qcols:
    col_name = qcol
    if i < len(names):
      col_name = names[i]
    quantiles[col_name] = data[qcol][0]
    i+=1
  return pd.DataFrame(quantiles)

# - quantiles: a dataframe where each column contains the quantiles of one
#   data set. The index (i.e. row names) of the dataframe is the quantile. The
#   column names are the names of the data set.
def plot_cdfs(quantiles, xlab="Value", ylab="CDF",
              legend_title="dataset", labels=[],
              interactive=False,
              title=''):
  dfs = []
  label = legend_title
  yval = range(quantiles.shape[0])
  esp = 1.0/(len(quantiles)-1)
  yval = [y * esp for y in yval]
  while label == xlab or label == ylab:
    label += '_'
  for col_idx, col in enumerate(quantiles.columns):
    col_label = col
    if col_idx < len(labels):
      col_label = labels[col_idx]
    dfs.append(pd.DataFrame({
        label: col_label,
        xlab: quantiles[col],
        ylab: yval
    }))
  cdfs = pd.concat(dfs)
  lines = alt.Chart(cdfs).mark_line().encode(
    # If you can draw a CDF, it has to be continuous real-valued
    x=xlab+":Q",
    y=ylab+":Q",
    color=label+":N"
  ).properties(
    title=title
  )
  if not interactive:
    return lines
  # Create a selection that chooses the nearest point & selects based on x-value
  nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=[ylab], empty='none')
  # Transparent selectors across the chart. This is what tells us
  # the y-value of the cursor
  selectors = alt.Chart(cdfs).mark_point().encode(
    y=ylab+":Q",
    opacity=alt.value(0),
  ).properties(
    selection=nearest
  )

  # Draw text labels near the points, and highlight based on selection
  text = lines.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest,
                       alt.Text(xlab+":Q", format=".2f"),
                       alt.value(' '))
  )

  # Draw a rule at the location of the selection
  rules = alt.Chart(cdfs).mark_rule(color='gray').encode(
    y=ylab+":Q",
  ).transform_filter(
    nearest.ref()
  )
  # Draw points on the line, and highlight based on selection
  points = lines.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
  )
  # Put the five layers into a chart and bind the data
  return alt.layer(lines, selectors, rules, text, points).interactive(
      bind_y=False)

# Functions to create the query

def query_machine_capacity(cell):
  return '''
SELECT machine_id, MAX(capacity.cpus) AS cpu_cap,
  MAX(capacity.memory) AS memory_cap
FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.machine_events
GROUP BY 1
  '''.format(cell=cell)

def query_top_level_instance_usage(cell):
  return '''
SELECT CAST(FLOOR(start_time/(1e6 * 300)) * (1000000 * 300) AS INT64) AS time,
  collection_id,
  instance_index,
  machine_id,
  average_usage.cpus AS cpu_usage,
  average_usage.memory AS memory_usage
FROM `google.com:google-cluster-data`.clusterdata_2019_{cell}.instance_usage
WHERE (alloc_collection_id IS NULL OR alloc_collection_id = 0)
  AND (end_time - start_time) >= (5 * 60 * 1e6)
  '''.format(cell=cell)

def query_machine_usage(cell):
  return '''
SELECT u.time AS time,
  u.machine_id AS machine_id,
  SUM(u.cpu_usage) AS cpu_usage,
  SUM(u.memory_usage) AS memory_usage,
  MAX(m.cpu_cap) AS cpu_capacity,
  MAX(m.memory_cap) AS memory_capacity
FROM ({instance_usage}) AS u JOIN
 ({machine_capacity}) AS m
ON u.machine_id = m.machine_id
GROUP BY 1, 2
  '''.format(instance_usage = query_top_level_instance_usage(cell),
             machine_capacity = query_machine_capacity(cell))

def query_machine_utilization_distribution(cell):
  return '''
SELECT APPROX_QUANTILES(IF(cpu_usage > cpu_capacity, 1.0, cpu_usage / cpu_capacity), 100) AS cpu_util_dist,
  APPROX_QUANTILES(IF(memory_usage > memory_capacity, 1.0, memory_usage / memory_capacity), 100) AS memory_util_dist
FROM ({table})
  '''.format(table = query_machine_usage(cell))

cell = 'c' #@param ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
query = query_machine_utilization_distribution(cell)
machine_util_dist = client.query(query).to_dataframe()
plot_cdfs(pick_quantiles_from_wide_dataframe(machine_util_dist, ['cpu_util_dist', 'memory_util_dist'], ['CPU', 'Memory']), xlab='x - resource utilization (%)', ylab="Probability (resource utilization < x)", interactive=True)

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)